<a href="https://colab.research.google.com/github/wonderfulvamsi/-My_1st_ML_Project/blob/master/DeepDreamApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%writefile DeepDreamApp.py

import torch
from torchvision import models, transforms
import numpy as np
#from matplotlib import pyplot

from PIL import Image, ImageFilter, ImageChops

import streamlit as st

#--------------------------------------FrontEnd

st.title('Deep Dream <3')
st.write('This lets you to visualize what each layer of your DNN learnt to recognize')
st.markdown('we use VGG16 as default in this app')

IMAGE_PATH = st.file_uploader("Upload Content image...", type="jpg")

if IMAGE_PATH is not None:
    st.image(IMAGE_PATH, caption='Uploaded Content Image.', use_column_width=True)
    img = Image.open(IMAGE_PATH)

# Deep dream configs

LAYER_ID = 28 # The layer to maximize the activations through

NUM_ITERATIONS = 5 # Number of iterations to update the input image with the layer's gradient

LR = 0.2

# We downscale the image recursively, apply the deep dream computation, scale up, and then blend with the original image 
# to achieve better result.
NUM_DOWNSCALES = 20

BLEND_ALPHA = 0.6

CUDA_ENABLED = False

img_size = None
Model = None

if st.button("Run on my own model"):
  img_size=st.number_input('Image size',min_value=1,max_value=2000,step=1)
  Model=st.file_uploader("Upload your costume PyTorch model",type=['pt','pth'])
  Model=torch.load(Model)

if img_size is None:
  img_size = 224

if Model is None:
  Model=models.vgg16(pretrained=True)

if Model is not None:
  st.write("Model Downloaded...")

modules = list(Model.features.modules())
st.write("Layers in the model..")

layerno=0

for l in modules[0]:
  layerno+=1
  st.write("Layer",layerno," ",l)

layer_to_dream = st.slider('Select the layer',min_value=1,max_value=(len(modules)-1),step=1)



class DeepDream:
    def __init__(self, image):
        self.image = image
        self.model = Model
        if CUDA_ENABLED:
            self.model = self.model.cuda()
        self.modules = list(self.model.features.modules())
        
                # vgg16 use 224x224 images
        imgSize = img_size
        self.transformMean = [0.485, 0.456, 0.406]
        self.transformStd = [0.229, 0.224, 0.225]
        self.transformNormalise = transforms.Normalize(
            mean=self.transformMean,
            std=self.transformStd
        )
        
        self.transformPreprocess = transforms.Compose([
            transforms.Resize((imgSize, imgSize)),
            transforms.ToTensor(),
            self.transformNormalise
        ])
        
        self.tensorMean = torch.Tensor(self.transformMean)
        if CUDA_ENABLED:
            self.tensorMean = self.tensorMean.cuda()

        self.tensorStd = torch.Tensor(self.transformStd)
        if CUDA_ENABLED:
            self.tensorStd = self.tensorStd.cuda()

    def toImage(self, input):
        return input * self.tensorStd + self.tensorMean


class DeepDream(DeepDream):
    def deepDream(self, image, layer, iterations, lr):
        transformed = self.transformPreprocess(image).unsqueeze(0)
        if CUDA_ENABLED:
            transformed = transformed.cuda()
        input = torch.autograd.Variable(transformed, requires_grad=True)
        self.model.zero_grad()
        for _ in range(iterations):
            out = input
            for layerId in range(layer):
                out = self.modules[layerId + 1](out)
            loss = out.norm()
            loss.backward()
            input.data = input.data + lr * input.grad.data

        input = input.data.squeeze()
        input.transpose_(0,1)
        input.transpose_(1,2)
        input = np.clip(self.toImage(input), 0, 1)
        return Image.fromarray(np.uint8(input*255))


class DeepDream(DeepDream):
    def deepDreamRecursive(self, image, layer, iterations, lr, num_downscales):
        if num_downscales > 0:
            # scale down the image
            image_small = image.filter(ImageFilter.GaussianBlur(2))
            small_size = (int(image.size[0]/2), int(image.size[1]/2))            
            if (small_size[0] == 0 or small_size[1] == 0):
                small_size = image.size
            image_small = image_small.resize(small_size, Image.ANTIALIAS)
            # run deepDreamRecursive on the scaled down image
            image_small = self.deepDreamRecursive(image_small, layer, iterations, lr, num_downscales-1)
            
            # Scale up the result image to the original size
            image_large = image_small.resize(image.size, Image.ANTIALIAS)
            
            # Blend the two image
            image = ImageChops.blend(image, image_large, BLEND_ALPHA)
        img_result = self.deepDream(image, layer, iterations, lr)
        img_result = img_result.resize(image.size)
        return img_result

    def deepDreamProcess(self, layerID):
        return self.deepDreamRecursive(self.image, layerID, NUM_ITERATIONS, LR, NUM_DOWNSCALES)

def Dreaming(layerID):
   img_deep_dream = DeepDream(img).deepDreamProcess(layerID)

   st.image(img_deep_dream,caption='Dreamt image..',use_column_width=True)

if st.button("Submit"):
  Dreaming(layer_to_dream)

Overwriting DeepDreamApp.py


In [2]:
!pip install streamlit -q

In [3]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

--2020-07-03 13:47:06--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.84.169.173, 52.54.253.53, 107.23.162.152, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.84.169.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.15’

ngrok-stable-linux- 100%[===================>]  13.13M  36.0MB/s    in 0.4s    

2020-07-03 13:47:06 (36.0 MB/s) - ‘ngrok-stable-linux-amd64.zip.15’ saved [13773305/13773305]

replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [4]:
get_ipython().system_raw('./ngrok http 8501 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://ff0387b0bc5f.ngrok.io


In [ ]:
!streamlit run DeepDreamApp.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.67.253.62:8501

